<a href="https://colab.research.google.com/github/TiagoIesbick/dashboard-etl/blob/main/settled_last_month.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
%%capture
!pip install tabula-py[jpype]
import pandas as pd
import pathlib
import re
from tabula import read_pdf

In [14]:
def str_to_float(value: str) -> float:
  value = str(value).replace('.', '').replace(',', '.')
  value = re.sub(r'[A-Za-z]|\s', '', value)
  return float(0) if value == '' else float(value)

def agrupamento_format(value: str) -> str:
  if re.search(r'^0{2}', value[:2]):
    value = '   ' + value
  elif re.search(r'^[1-9]\d', value[:2]):
    value = '      ' + value
  return value

In [63]:
folder = pathlib.Path(r'/content/drive/MyDrive/settled_last_month')

df_all = pd.DataFrame()

for pdf_file in folder.glob('*.pdf'):
  dates = read_pdf(pdf_file, area=[100, 71, 100+16, 71+200], pages=1, pandas_options={'header': None})[0]
  fonte = read_pdf(pdf_file, area=[115, 71, 115+20, 71+245], pages=1, pandas_options={'header': None})[0]
  fonte = re.search(r'[\d\.]*', fonte.iloc[0,1]).group()
  df = read_pdf(pdf_file, area=[180, 71, 180+370, 71+755], guess=False, pandas_options={'header': [0]}, pages='all', multiple_tables=False, encoding='latin1')[0]
  df.drop(df.loc[df['Agrupamento']=='Agrupamento'].index, inplace=True)
  df_out = df.loc[df['%'].isna()]
  for i in df_out.index:
    df.loc[i-1, 'Agrupamento'] += ' ' + df.loc[i, 'Agrupamento']
  df.drop(df_out.index, inplace=True)
  if len(df.loc[df['Agrupamento']=='Total']) > 1:
    df.drop(df.loc[df['Agrupamento']=='Total'].index[1:], inplace=True)
  df.reset_index(drop=True, inplace=True)
  df.iloc[:, 1:] = df.iloc[:, 1:].map(str_to_float)

  df['unidorc'] = int(re.sub(r'\D', '', df.iloc[1,0]))
  df['fonte'] = fonte
  li = list(df.loc[df['Agrupamento'].str.startswith('00')].index)
  li.append(df.index[-1])
  for i,j in enumerate(li):
    subacao = int(re.sub(r'\D', '',df.loc[j, 'Agrupamento']))
    if i+1 < len(li):
      idx = li[i]
      while idx <= li[i+1]:
        df.loc[idx, 'subacao'] = subacao
        idx += 1
  df.loc[~df['Agrupamento'].str.startswith(('0', 'T')), 'elemento'] = df['Agrupamento'].str.replace(r'\D', '', regex=True)
  # display(df.loc[~df['Agrupamento'].str.startswith(('T', '0')), ['Atualizado', 'Pré-Empenhado', 'Empenhado', 'Disponível', 'Liquidado', 'Pago', 'A Liquidar', 'A Pagar']].sum())
  df_all = pd.concat([df_all, df], ignore_index=True)

df_all['Agrupamento'] = df_all['Agrupamento'].map(agrupamento_format)

In [64]:
df_cubo_7001 = pd.read_excel(r'/content/drive/MyDrive/cubo/cubo.xlsx', sheet_name='7001', converters={'elemento': str})
df_cubo_7002 = pd.read_excel(r'/content/drive/MyDrive/cubo/cubo.xlsx', sheet_name='7002', converters={'elemento': str})
df_cubo_7003 = pd.read_excel(r'/content/drive/MyDrive/cubo/cubo.xlsx', sheet_name='7003', converters={'elemento': str})

In [65]:
df_7001 = df_cubo_7001.merge(
    df_all[
        ['Agrupamento', 'unidorc', 'subacao', 'elemento', 'fonte', 'Liquidado']
        ].rename(
            {'Liquidado': f'Liquidado em {dates.iloc[0,1]}'}, axis=1
            ), how='left', on=['Agrupamento', 'unidorc', 'subacao', 'elemento', 'fonte'])

df_7002 = df_cubo_7002.merge(
    df_all[
        ['Agrupamento', 'unidorc', 'subacao', 'elemento', 'fonte', 'Liquidado']
        ].rename(
            {'Liquidado': f'Liquidado em {dates.iloc[0,1]}'}, axis=1
            ), how='left', on=['Agrupamento', 'unidorc', 'subacao', 'elemento', 'fonte'])

df_7003 = df_cubo_7003.merge(
    df_all[
        ['Agrupamento', 'unidorc', 'subacao', 'elemento', 'fonte', 'Liquidado']
        ].rename(
            {'Liquidado': f'Liquidado em {dates.iloc[0,1]}'}, axis=1
            ), how='left', on=['Agrupamento', 'unidorc', 'subacao', 'elemento', 'fonte'])

In [66]:
with pd.ExcelWriter(r'/content/drive/MyDrive/cubo/cubo_with_settled_last_month.xlsx') as writer:
  df_7001.to_excel(writer, sheet_name='7001', index=False)
  df_7002.to_excel(writer, sheet_name='7002', index=False)
  df_7003.to_excel(writer, sheet_name='7003', index=False)